In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import tabulate
from tabulate import tabulate
import pandas as pd

We begin with the entire transient heat conduction equation in radial coordinates:
$$\frac{\delta T}{\delta t} = \alpha \nabla^2 T = \alpha[\frac{1}{r} \frac{\delta}{\delta r}(r \frac{\delta T}{\delta r}) + \frac{1}{r^2} \frac{\delta^2 T}{\delta \theta^2} + \frac{\delta^2 T}{\delta z^2}]$$
We can now simpify the equation using the problem statement and some assumptions. We are going to assume a constant heat transfer coeffient and uniform step size in the time and radial dimenisions respectively. Now we are going to use the problem statement to state that the carrot acts as a long cylinder which means we are going to neglect heat transfer in the axial direction and anziotropic heat conduction which means that with a uniform hot water bath we can expect that there will be 0 temperature gradient along the theta direction which means that we can get the equation:
$$\frac{\delta T}{\delta t} = \alpha[\frac{\delta^2 T}{\delta r^2} + \frac{1}{r} (\frac{\delta T}{\delta r})]$$
from here we should replace the left side of the above equation with a two point forward difference approximation for time, and the right side of the equation with 3 point central difference approximations for the first and second derivatives. The function named centralDifCoef below will output the coefients for a central finite difference approximation for an given derivative, and order of convergence. The 3 point method listed in the problem statement has a second order of convergence, but as we will show in the experimental design discussion this is likely insuffient for the radial approximation which means our bottleneck for accuracy can likely be improved by using more points for our radial approximations. 

In [4]:
#returns the coef for evenly spaced points with central finite difference method
def centralDifCoef(derivative, order):
    p = int((2* np.floor((derivative+1)/2) -2 +order)/2)
    A = np.zeros((2*p+1, 2*p+1))
    A[0] = 1
    for i in range(1, 2*p+1):
        for j in range(0, p):
            A[i,j] = (-1*p + j)**i
        temp = p-1
        for j in range(p+1, 2*p+1):
            A[i,j] = (p-temp)**i
            temp = temp - 1
    
    b = np.zeros(2*p+1)
    b[derivative] = np.math.factorial(derivative)
    c = np.linalg.solve(A,b)
    return c

In [7]:
print(centralDifCoef(1,2))
print(centralDifCoef(2,2))

[-0.5  0.   0.5]
[ 1. -2.  1.]


We can now replace the the differentials in the simplified equation with the following where the radial steps are indexed by i and the time steps are indexed by n:
$$\frac{T_i^{n+1}-T_i^n}{\Delta t} = \alpha \cdot (\frac{T_{i-1}^n - 2\cdot T_i^n + T_{i+1}^n}{\Delta r^2} + \frac{1}{r_i} \cdot (\frac{T_{i-1}^n - T_{i+1}^n}{2 \cdot \Delta r}))$$
We now set $d = \frac{\alpha \Delta t}{\Delta r^2}$ and $c = \frac{\alpha \Delta t}{2 \Delta r}$ for notational convience
so the final expression we can use to solve for points iteratively is:
$$T_i^{n+1} = T_i^n + d \cdot (T_{i-1}^n-2 \cdot T_i^n + T_{i+1}^n) + \frac{c}{r_i} \cdot (T_{i-1}^n - T_{i+1}^n)$$
it is now important to note that our steps are considered from the outside moving in, i.e. $T_{5}^2$ is the temperature at 5 steps in from the ghost node towards the center of the carrot at the second time step

In [137]:
def timeStep(previous, alpha, deltaT, deltaR, radius):
    leftShift = previous[1:] #This is done in vector notation so that we can run it in parellel to make the best use of LAPACK
    leftShift = np.append(leftShift,0)
    rightShift = previous[0:-1]
    rightShift = np.insert(rightShift,0,0)
    nextStep = previous + (alpha*deltaT)/deltaR**2 * (rightShift - 2*previous + leftShift) + (alpha*deltaT/(2*deltaR*radius))*\
    (rightShift - leftShift)
    nextStep[0] = (50*(373-nextStep[1]) *deltaR/0.569) + nextStep[1]  #set boundary conditions that hold for any time
    nextStep[-1] = nextStep[-2] #this is the inner BC we set flux equal to 0 so the GN is equal to the last point
    return(nextStep)

In [152]:
data = np.zeros((1,102)) #choosing to use 100 points in the carrot, plus 2 ghost nodes
data[:] = 277
data[0,0] = 373
alpha = 0.569/(1029*3801)
deltaT = 10**-2
deltaR = 0.0075/100
radius = np.linspace(0.0075*1.01,0,102)
radius[-1] = 1 #this is a dummy value for the ghost node in the center which is delt with via the BC in the above function
previous = data[0,:]
for i in range(0,16):
    for j in range(0,6000):
        nex = timeStep(previous, alpha, deltaT, deltaR, radius)
        data = np.vstack((data,nex))
        previous = nex
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


For problem E we just need to change the intial value and the BC for the outer ghost node

In [160]:
def timeStepBoulder(previous, alpha, deltaT, deltaR, radius):
    leftShift = previous[1:] #This is done in vector notation so that we can run it in parellel to make the best use of LAPACK
    leftShift = np.append(leftShift,0)
    rightShift = previous[0:-1]
    rightShift = np.insert(rightShift,0,0)
    nextStep = previous + (alpha*deltaT)/deltaR**2 * (rightShift - 2*previous + leftShift) + (alpha*deltaT/(2*deltaR*radius))*\
    (rightShift - leftShift)
    nextStep[0] = (50*(368-nextStep[1]) *deltaR/0.569) + nextStep[1]  #set boundary conditions that hold for any time
    nextStep[-1] = nextStep[-2] #this is the inner BC we set flux equal to 0 so the GN is equal to the last point
    return(nextStep)

In [ ]:
dataBoulder = np.zeros((1,102)) #choosing to use 100 points in the carrot, plus 2 ghost nodes
dataBoulder[:] = 277
dataBoulder[0,0] = 368
alpha = 0.569/(1029*3801)
deltaT = 10**-2
deltaR = 0.0075/100
radius = np.linspace(0.0075*1.01,0,102)
radius[-1] = 1 #this is a dummy value for the ghost node in the center which is delt with via the BC in the above function
previous = data[0,:]
for i in range(0,16):
    for j in range(0,6000):
        nex = timeStep(previous, alpha, deltaT, deltaR, radius)
        dataBoulder = np.vstack((dataBoulder,nex))
        previous = nex
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [158]:
for i in range(60000,90000):
    if data[i,-1] >= 358:
        print("index is:", i, "center temperature is:", data[i,-1])
        break

index is: 68648 center temperature is: 358.0004102195146


#### Biggest question I've got for the time being is how to deal with the central BC. We can introduce the outer ghost node in the radial direction as the 0th point and then set it equal to the ambient temperature of the water, 100C. I am not sure how to do the same thing for the center. The IC is that T(r,t = 0) = 4C for all r

a discussion on analog sample rates for thermocouples:
https://www.embedded.com/sampling-rates-for-analog-sensors/
an example of an inexpensive thermoresistor that can be used with an RPi or Ardunio.
https://www.adafruit.com/product/372
The key point here is that it is completely realistic to expect sample sizes on the order of $O(10^{-6})-O(10^{-4})$ seconds depending on the clock rate of our controller and with a bit of smoothing we can expect good accuracy. With some work we can probably get this at least a couple of orders of magnitude lower, because most wire formats are capable of transmitting at least 1Mbit/s which means getting a 16 bit sample in under 0.1ms should be no problem as a default. If we want 1Gbit/s we can likely use a LAN hookup and then translate the wire format in the controller which would give us 16 bits in under $0.1 \mu s$. We can look at I2C data transfer rates and see that we will not be bottlenecked by our wire format for this type of application. This means we can expect the number of significant digits in the temperature readings to come from the resistor. To gather experimental data to see if our readings are correct the best way I can think of right now is to take a paddle bit and drill out the center of the carrot and then stick our thermocouple to the inside of the cylinder. This means that our experimental data will be constrained by the accuracy of the calaper that we use to measure the thickness of the carrot, and how many different sized paddle bits we can find. We are going to be lucky if we can get millimeter precison here, likely we will have $O(10^{-3})-O(10^{-2})$ meter precision here. Using our current methods with a 2 point forward difference approximation for time and a 3 point central difference method for radius we can see that the error is bounded above by $O(\Delta t) + O(\Delta r^2)$ which means that we expect that it will be easier to design this experiment in a way where the error from the radial term dominates. This is easy to remidy by using a higher order central difference method for the interior points such as a 5 point method which will result in 4th order convergence. At any level of desired accuracy we should probably use a finite difference method with 3 time faster convergence for the radial term vs. the time term.

In [8]:
print(centralDifCoef(1,4))
print(centralDifCoef(2,4))

[ 0.08333333 -0.66666667  0.          0.66666667 -0.08333333]
[-0.08333333  1.33333333 -2.5         1.33333333 -0.08333333]


In [89]:
np.linspace(0.0075*1.1,-0.00075,13)

array([ 8.25000000e-03,  7.50000000e-03,  6.75000000e-03,  6.00000000e-03,
        5.25000000e-03,  4.50000000e-03,  3.75000000e-03,  3.00000000e-03,
        2.25000000e-03,  1.50000000e-03,  7.50000000e-04, -1.73472348e-18,
       -7.50000000e-04])

In [135]:
radius = np.linspace(0.0075*1.01,-0.0075*0.01,103)
radius[-2]=0
radius

array([ 7.575e-03,  7.500e-03,  7.425e-03,  7.350e-03,  7.275e-03,
        7.200e-03,  7.125e-03,  7.050e-03,  6.975e-03,  6.900e-03,
        6.825e-03,  6.750e-03,  6.675e-03,  6.600e-03,  6.525e-03,
        6.450e-03,  6.375e-03,  6.300e-03,  6.225e-03,  6.150e-03,
        6.075e-03,  6.000e-03,  5.925e-03,  5.850e-03,  5.775e-03,
        5.700e-03,  5.625e-03,  5.550e-03,  5.475e-03,  5.400e-03,
        5.325e-03,  5.250e-03,  5.175e-03,  5.100e-03,  5.025e-03,
        4.950e-03,  4.875e-03,  4.800e-03,  4.725e-03,  4.650e-03,
        4.575e-03,  4.500e-03,  4.425e-03,  4.350e-03,  4.275e-03,
        4.200e-03,  4.125e-03,  4.050e-03,  3.975e-03,  3.900e-03,
        3.825e-03,  3.750e-03,  3.675e-03,  3.600e-03,  3.525e-03,
        3.450e-03,  3.375e-03,  3.300e-03,  3.225e-03,  3.150e-03,
        3.075e-03,  3.000e-03,  2.925e-03,  2.850e-03,  2.775e-03,
        2.700e-03,  2.625e-03,  2.550e-03,  2.475e-03,  2.400e-03,
        2.325e-03,  2.250e-03,  2.175e-03,  2.100e-03,  2.025e

In [143]:
i

72778